In [1]:
import warnings, urllib3, requests, pandas as pd, re, os
from bs4 import BeautifulSoup
from io import StringIO

# -- 1. Pobranie strony (HTTP, wyłączamy SSL-warn) -----------------------------
warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

URL = "http://www.tenis.centrumwarszawianka.pl/tenis.html"   # bez #cennik!
HEADERS = {"User-Agent": "Mozilla/5.0"}

resp = requests.get(URL, headers=HEADERS, timeout=15, verify=False)
resp.encoding = "utf-8"                      # <-- klucz przeciwko „krzaczkom”
html = resp.text

# -- 2. Bierzemy pierwszą tabelę ------------------------------------------------
table_html = BeautifulSoup(html, "lxml").find("table")
df = pd.read_html(StringIO(str(table_html)), header=None)[0]   # FutureWarning gone

df.columns = ["raw", "outdoor", "indoor"]          # 3 kolumny – reszta pusta
df = df.dropna(subset=["raw"]).reset_index(drop=True)

# -- 3. Odrzucamy banery „Sezon LETNI …” ---------------------------------------
df = df[~df["raw"].str.contains("Sezon", case=False, na=False)]

# -- 4. Tworzymy kolumnę 'sekcja' (Pon-Pt / So-Nd) -----------------------------
sec_mask = df["raw"].str.contains(r"Poniedziałek|Sobota", case=False, na=False)
df.loc[sec_mask, "sekcja"] = df.loc[sec_mask, "raw"]
df["sekcja"] = df["sekcja"].ffill()
df = df[~sec_mask]                              # zostają same wiersze z godzinami

# -- 5. Czyścimy liczby ---------------------------------------------------------
def pln2int(x: str) -> int:
    return int(re.sub(r"[^0-9]", "", str(x)) or 0)

df["outdoor_zł"] = df["outdoor"].map(pln2int)
df["indoor_zł"]  = df["indoor"].map(pln2int)
df = df[["sekcja", "raw", "outdoor_zł", "indoor_zł"]]
df.rename(columns={"raw": "godziny"}, inplace=True)

print(df)

# -- 6. Zapis -------------------------------------------------------------------
out = "cennik_warszawianka_letni_2025.csv"
df.to_csv(out, index=False, encoding="utf-8")
print("✔ Zapisano:", os.path.abspath(out))



                  sekcja        godziny  outdoor_zł  indoor_zł
2  Poniedziałek - Piątek  07:00 - 12:00          70         80
3  Poniedziałek - Piątek  12:00 - 15:00          60         70
4  Poniedziałek - Piątek  15:00 - 22:00          90        100
5  Poniedziałek - Piątek  22:00 - 24:00          75         85
7     Sobota - Niedziela  07:00 - 24:00          80         90
✔ Zapisano: C:\Users\User\cennik_warszawianka_letni_2025.csv


In [9]:
import pandas as pd
df=pd.read_csv("cennik_warszawianka_letni_2025.csv")

In [11]:
df.head()

,sekcja,godziny,outdoor_zł,indoor_zł
0,Poniedziałek - Piątek,07:00 - 12:00,70,80
1,Poniedziałek - Piątek,12:00 - 15:00,60,70
2,Poniedziałek - Piątek,15:00 - 22:00,90,100
3,Poniedziałek - Piątek,22:00 - 24:00,75,85
4,Sobota - Niedziela,07:00 - 24:00,80,90
